# DINOv3 Similarity Visualization

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/DINOv3_visualization.ipynb)

This notebook demonstrates the new visualization functionality for DINOv3 similarity analysis in the geoai package.

## Install packages

To use the new functionality, ensure the required packages are installed.

In [ ]:
# %pip install geoai-py

## Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import geoai
from geoai.dinov3 import DINOv3GeoProcessor, visualize_similarity_results

%matplotlib inline

In [ ]:
url = "https://huggingface.co/spaces/pszemraj/dinov3-viz-sat493m/resolve/main/aralsea_tmo_2000238_lrg.jpg"
test_image_path = geoai.download_file(url)

In [ ]:
# Create and display test image
test_image = Image.open(test_image_path)

plt.figure(figsize=(8, 8))
plt.imshow(test_image)
plt.axis("off")
plt.show()

## Method 1: Quick Visualization with visualize_similarity_results()

The easiest way to create and visualize similarity results in one step.

In [ ]:
# Query coordinates
query_coords = (1250, 1250)

# Side-by-side visualization
results = visualize_similarity_results(
    input_image=test_image_path,
    query_coords=query_coords,
    overlay=False,  # Side-by-side view
    figsize=(15, 6),
    colormap="turbo",
    show_query_point=True,
)

plt.show()
print(f"Query patch coordinates: {results['patch_coords']}")
print(f"Patch grid size: {results['patch_grid_size']}")

In [ ]:
# Overlay visualization
results_overlay = visualize_similarity_results(
    input_image=test_image_path,
    query_coords=query_coords,
    overlay=True,  # Overlay view
    alpha=0.6,  # Transparency
    colormap="jet",
    show_query_point=True,
)

plt.show()

## Method 2: Step-by-step with DINOv3GeoProcessor

For more control over the process, you can use the processor methods directly.

In [ ]:
# Initialize processor
processor = DINOv3GeoProcessor()

# Load and preprocess image
data, metadata = processor.load_image(test_image_path)
image = processor.preprocess_image_for_dinov3(data)
features, h_patches, w_patches = processor.extract_features(image)

print(f"Image size: {image.size}")
print(f"Features shape: {features.shape}")
print(f"Patch grid: {h_patches} x {w_patches}")

In [ ]:
# Convert pixel coordinates to patch coordinates
img_w, img_h = data.shape[2], data.shape[1]
patch_x = int((query_coords[0] / img_w) * w_patches)
patch_y = int((query_coords[1] / img_h) * h_patches)

print(f"Query pixel coordinates: {query_coords}")
print(f"Query patch coordinates: ({patch_x}, {patch_y})")

# Compute similarity
similarities = processor.compute_patch_similarity(features, patch_x, patch_y)
similarity_array = similarities.cpu().numpy()

print(f"Similarity range: {similarity_array.min():.3f} - {similarity_array.max():.3f}")

In [ ]:
# Visualize patch grid
fig = processor.visualize_patches(
    image=image, features=features, patch_coords=(patch_x, patch_y), figsize=(12, 8)
)

plt.show()

In [ ]:
# Create side-by-side visualization
fig = processor.visualize_similarity(
    source=test_image_path,
    similarity_data=similarity_array,
    query_coords=query_coords,
    patch_coords=(patch_x, patch_y),
    overlay=False,
    colormap="jet",
    figsize=(15, 6),
)

plt.show()

In [ ]:
# Create overlay visualization
fig = processor.visualize_similarity(
    source=test_image_path,
    similarity_data=similarity_array,
    query_coords=query_coords,
    overlay=True,
    alpha=0.5,
    colormap="jet",
)

plt.show()

## Method 3: Create Overlay Image Array

You can also create overlay images programmatically for further processing.

In [ ]:
# Create overlay as numpy array
overlay_img = processor.create_similarity_overlay(
    source=test_image_path, similarity_data=similarity_array, colormap="jet", alpha=0.6
)

# Display the overlay
plt.figure(figsize=(10, 10))
plt.imshow(overlay_img)
plt.title("Programmatically Created Overlay")
plt.axis("off")
plt.show()

# Save overlay as image file
overlay_pil = Image.fromarray((overlay_img * 255).astype(np.uint8))
overlay_pil.save("demo_overlay.png")
print("Overlay saved to demo_overlay.png")

## Test Different Query Points

Let's test similarity analysis with different query points.

In [ ]:
# Test multiple query points
query_points = [
    (128, 128),  # Red circle
    (384, 128),  # Green circle
    (128, 384),  # Blue circle
    (384, 384),  # Yellow circle
]

colors = ["Red", "Green", "Blue", "Yellow"]

fig, axes = plt.subplots(2, 2, figsize=(20, 16))
axes = axes.flatten()

for i, (coords, color) in enumerate(zip(query_points, colors)):
    # Quick visualization for each point
    results = visualize_similarity_results(
        input_image=test_image_path,
        query_coords=coords,
        overlay=True,
        alpha=0.6,
        show_query_point=True,
    )

    # Close the generated figure and create our own subplot
    plt.close(results["visualization"])

    # Get the similarity data and create our own visualization
    similarity_data = results["image_dict"]["image"][0]
    data, _ = processor.load_image(test_image_path)
    display_img = np.transpose(data, (1, 2, 0))

    # Show on subplot
    axes[i].imshow(display_img / 255.0)
    im = axes[i].imshow(similarity_data, cmap="turbo", alpha=0.6, vmin=0, vmax=1)
    axes[i].plot(
        coords[0],
        coords[1],
        "r*",
        markersize=15,
        markeredgecolor="white",
        markeredgewidth=2,
    )
    axes[i].set_title(f"Query: {color} Circle at {coords}")
    axes[i].axis("off")

    # Add colorbar
    plt.colorbar(im, ax=axes[i], fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()